In [53]:
from torch.utils.data import Dataset, DataLoader
from dataloader import MRIDataset

In [54]:
dataset = MRIDataset(train=False)
dataloader = DataLoader(dataset = dataset, batch_size= 10, shuffle= True, num_workers=2)

In [55]:
dataiter = iter(dataloader)
data = dataiter.next()
features, labels = data
print(features.shape)

torch.Size([10, 1, 14, 240, 240])
